# Example of saving data to GEFF

This example illustrates how we can save the tracked data into a GEFF (Graph Exchange File Format).

In [2]:
from laptrack import LapTrack
from laptrack import data_conversion
from laptrack import datasets
import geff

In [3]:
lt = LapTrack(
    metric="sqeuclidean",
    cutoff=15**2,
    splitting_metric="sqeuclidean",
    merging_metric="sqeuclidean",
    splitting_cutoff=15**2,
    merging_cutoff=15**2,
)

In [7]:
coordinate_cols = ["position_x", "position_y"]
frame_col = "frame"

In [9]:
spots_df = datasets.simple_tracks()
track_df, split_df, merge_df = lt.predict_dataframe(
    spots_df,
    coordinate_cols=coordinate_cols,
    frame_col=frame_col,
    only_coordinate_cols=False,
)

In [10]:
tree_geff = data_conversion.convert_dataframes_to_geff_networkx(
    track_df,
    split_df,
    merge_df,
    coordinate_cols=coordinate_cols,
    frame_col=frame_col,
)

In [11]:
tree_geff.nodes[0]

{'frame': 0,
 'position_x': np.float64(178.4125746427294),
 'position_y': np.float64(185.18866073671265)}

In [14]:
geff.write_nx(tree_geff, "/tmp/tree.geff")
geff.validate("/tmp/tree.geff")